# Prepare necessary metadata files from TLG library

In [ ]:
import os, re, string
from pprint import pprint
from itertools import takewhile

REF = os.path.expanduser('~/github/pthu/tfbuilder/tlg_to_tf')

f1 = open(f'{REF}/data/refs_1.txt', encoding='utf-8')
f2 = open(f'{REF}/data/refs_2.txt', encoding='utf-8')

In [ ]:
def splitLine(line):
    if line.startswith('    '):
        return (False, line.strip())
    else:
        sline = line.strip()
        first_space = sline.find(' ')
        return (sline[:first_space].strip(), 
                sline[first_space:].strip())

def authorMetadata(file):
    author_dict = {}
    key = None
    authorFound = False
    for l in file.readlines():
        line = l.strip()
        if line.startswith('key') and len(line.split()) == 3:
            authorFound = False
        elif authorFound:
            code, content = splitLine(line)
            author_dict[key].update({code: content})
        elif line.startswith('key') and len(line.split()) == 2:
            authorFound = True
            _, key = line.split()
            author_dict[key] = {}
    return author_dict


def titleMetadata(file):
    workRE    = re.compile(r'^([0-9]+){4} (\w+){3}$')
    work_dict = {}
    start     = False
    workName  = ''
    nameFound = False
    workFound = False
    for l in file.readlines():
        line = l.strip()
        if len(line) == 4 and line.isdigit():
            workFound = False
        elif workRE.fullmatch(line):
            if start:
                work_dict[work] = (name, workName)
            start = True
            authorkey, workkey = tuple(line.split(' '))
            work = f'tlg{authorkey}-{workkey}'
            workFound = True
            nameFound = False
            workName  = ''
            continue
        elif workFound:
            if not nameFound:
                if ',' in line:
                    name = line[:line.find(',')]
                else:
                    name = line
                nameFound = True
                workName += line
            else:
                workName += ' ' + line        
        else:
            continue
    return work_dict


def workMetadata(file, title_data=titleMetadata(f1)):
    refs_dict   = {}
    authorFound = False
    workFound   = False
    CUR         = None

    for line in file.readlines():
        sline = line.strip()
        
        if sline == '':
            continue
            
        elif sline.startswith('key') and len(sline.split()) == 2:
            if not CUR == None:
                refs_dict[CUR].update(work_dict)
                refs_dict[CUR]['title'] = title_data[CUR][0] if CUR in title_data else 'not provided'
                refs_dict[CUR]['title_full'] = title_data[CUR][1] if CUR in title_data else 'not provided'
            author_dict = {}
            work_dict = {}
            authorFound = True
            workFound = False

        elif sline.startswith('key') and len(sline.split()) == 3:
#             pprint(refs_dict)
            if not CUR == None:
                refs_dict[CUR].update(work_dict)
                refs_dict[CUR]['title'] = title_data[CUR][0] if CUR in title_data else 'not provided'
                refs_dict[CUR]['title_full'] = title_data[CUR][1] if CUR in title_data else 'not provided'
            work_dict = {}
            code, author, work = sline.split()
            CUR = f'tlg{author}-{work}'
            refs_dict[CUR] = author_dict.copy()
            authorFound = False
            workFound = True
            
        if authorFound:
            cod1, cont1 = splitLine(line)
            if cod1:
                code1, content1 = cod1, cont1
            else:
                content1 = content1 + ' ' + cont1
            author_dict[code1] = content1
            
        elif workFound:
            cod2, cont2 = splitLine(line)
            if cod2:
                code2, content2 = cod2, cont2
            else:
                content2 = content2 + ' ' + cont2
            work_dict[code2] = content2
    
#     if CUR != None:
    refs_dict[CUR].update(work_dict)
    refs_dict[CUR]['title'] = title_data[CUR][0].title() if CUR in title_data else 'not provided'
    refs_dict[CUR]['title_full'] = title_data[CUR][1] if CUR in title_data else 'not provided'
    
    for i in refs_dict:
        if 'nam' in refs_dict[i]:
            refs_dict[i]['author'] = refs_dict[i].pop('nam').title()
        if 'edr' in refs_dict[i]:
            refs_dict[i]['editor'] = refs_dict[i].pop('edr').title()
        if 'geo' in refs_dict[i]:
            refs_dict[i]['geography'] = refs_dict[i].pop('geo')
        if 'dat' in refs_dict[i]:
            refs_dict[i]['date'] = refs_dict[i].pop('dat')
        if 'cit' in refs_dict[i]:
            refs_dict[i]['citation_scheme'] = refs_dict[i].pop('cit').lower()
        if 'wct' in refs_dict[i]:
            refs_dict[i]['tlg_wordcount'] = refs_dict[i].pop('wct')
        if 'wrk' in refs_dict[i]:
            refs_dict[i]['work'] = refs_dict[i].pop('wrk').title()
        if 'pub' in refs_dict[i]:
            refs_dict[i]['publisher'] = refs_dict[i].pop('pub').title()
        if 'pag' in refs_dict[i]:
            refs_dict[i]['pages'] = refs_dict[i].pop('pag')
        if 'epi' in refs_dict[i]:
            refs_dict[i]['epithet'] = refs_dict[i].pop('epi')
        if 'typ' in refs_dict[i]:
            refs_dict[i]['type'] = refs_dict[i].pop('typ')
        if 'epi' in refs_dict[i]:
            refs_dict[i]['epithet'] = refs_dict[i].pop('epi')
        if 'pyr' in refs_dict[i]:
            refs_dict[i]['publication_year'] = refs_dict[i].pop('pyr')
        if 'ryr' in refs_dict[i]:
            refs_dict[i]['republication_year'] = refs_dict[i].pop('ryr')
        if 'pla' in refs_dict[i]:
            refs_dict[i]['place'] = refs_dict[i].pop('pla').title()
        if 'tit' in refs_dict[i]:
            refs_dict[i]['tit'] = refs_dict[i].pop('tit').title()

    
    return refs_dict
        

# author_result = authorMetadata(f2)
# title_result = titleMetadata(f1)
# bibl_result = bibliography(f2)
work_result = workMetadata(f2)
# pprint(author_result)
# pprint(title_result)
# pprint(bibl_result)
pprint(work_result)


f1.close()
f2.close()

In [ ]:
# Works starting with 'X' don't have a citation system
s = set()

for i in refs_result:
    if 'cit' not in refs_result[i]:
        s.add(i)
pprint(s)

# Check whether other works don't have a citation system
for i in refs_result:
    if i not in s:
        if 'cit' not in refs_result[i]:
            print(i)

# Check whether every entry has a title ('tit' | 'wrk')
p = set()
for i in refs_result:
    if not 'tit' in refs_result[i] and not 'wrk' in refs_result[i]:
        p.add(i)
print(p)

In [ ]:
# Derive the citation scheme
refs_list = []

for i in refs_result:
    if not i[1].startswith('X'):
        refs_list.append(list(filter(None, refs_result[i]['cit'].lower().split('/'))))

pprint(refs_list)

In [ ]:

def tlge2csv(file_path, out_path, ref_dict, metadata_dict):
    fpath = os.path.expanduser(file_path)
    file_list = []
    if os.path.isdir(fpath):
        with os.scandir(fpath) as it:
            for entry in it:
                file_list.append(entry)
    elif os.path.isfile(fpath):
        file_list.append(fpath)
    else:
        print('It looks like something is wrong with the file_path')
    
    if out_path.startswith('~'):
        out_path = os.path.expanduser(out_path)
    try:
        os.makedirs(out_path) 
    except OSError:
        pass
    
    def cleanRef(ref):
        if ref.isdigit():
            ref_out = ref
        elif ref.isalpha():
            ref_out = ref
        else:
            if '_' in ref:
                ref = ref[:ref.find('_')]
            if set(string.ascii_letters) & set(ref):
                if ref[0].isalpha():
                    ref = ''.join(takewhile(lambda c: not c.isdigit(), ref))
                else:
                    ref = ''.join(takewhile(lambda c: not c.isalpha(), ref))
            ref_out = ''.join(c for c in ref if c.isalnum() or c == '-')
            if not ref_out.isalpha():
                ref_out = ref_out.rstrip(string.ascii_letters)
        return ref_out
    
    
    for file in file_list:
        print(file)
        with open(file) as source:
            n, ext    = os.path.splitext(file)
            if not ext == '.txt':
                continue
            name      = n.split('/')[-1]
            tlgNum    = (name[3:7], name[8:])
            workMeta  = metadata_dict[tlgNum]                  # tuple(title, full bibliographical reference)
            citScheme = tuple(filter(None, \
                          ref_dict[tlgNum]['cit'].split('/').lower())) # tuple(citlevel1, citlevel2, citlevel3, etc.)
#             print(f'workMetadata = {workMeta}')
#             print(f'citation scheme = {citScheme}')
            
            csv_output = open(f'{out_path}{name}.csv', 'w+')
            header = '\t'.join(citScheme) + '\t' + 'text' + '\n'
            csv_output.write(header)
            remainder = ''
    
            for line in (l.strip() for l in source.readlines() if not l.strip() == ''):
                line += remainder
                remainder = ''
                # The try-except construct is used because tlgu sometimes breaks the line unexpectedly
                try:
                    ref, text = line.split('\t', 1)
                except ValueError:
                    remainder = line

                clean_ref = tuple((cleanRef(r) for r in filter(None, ref.split('.'))))
                
                # Check whether the tlg citation scheme is equal to what we find in the file
                # If not equal the scheme is allinged to the right, as seems to be most appropriate
                if len(clean_ref) != len(citScheme):
                    start = len(clean_ref) - len(citScheme)
                    clean_ref = clean_ref[start:]
                
                split_ref = '\t'.join(clean_ref)
                csv_output.write(split_ref + '\t' + text.strip().replace('\t', ' ') + '\n')
            
            csv_output.close()
            
            
tlge2csv('~/github/tlgu-1/out/', '~/github/tlgu-1/out/csv/', refs_result, works_result)
# tlge2csv('~/github/tlgu-1/out/tlg0555-002.txt', '~/github/tlgu-1/out/csv/', refs_result, works_result)
# tlge2csv('~/github/tlgu-1/out/tlg0533-017.txt', '~/github/tlgu-1/out/csv/', refs_result, works_result)



In [ ]:
ref = '120_3'
index = ref.find('_')
ref = ref[:index]
print(ref)

In [ ]:
citScheme = ('chapter', 'section', 'line')
clean_ref = ('1', 'B', '2', '18', '2')

if len(clean_ref) != len(citScheme):
    start = len(clean_ref) - len(citScheme)
    clean_ref = clean_ref[start:]
    print(clean_ref)

In [ ]:
import string
ref = 'p1'
if set(string.ascii_letters) & set(ref):
    if ref[0].isalpha():
        ref = ''.join(next(c for c in ref if not c.isdigit()))
    else:
        ref = ''.join(next(c for c in ref if not c.isalpha()))
print(ref)

In [ ]:
import string
from itertools import takewhile

def cleanRef(ref):
        if ref.isdigit():
            ref_out = ref
        elif ref.isalpha():
            ref_out = ref
        else:
            if '_' in ref:
                ref = ref[:ref.find('_')]
            if set(string.ascii_letters) & set(ref):
                if ref[0].isalpha():
                    ref = ''.join(takewhile(lambda c: not c.isdigit(), ref))
                else:
                    ref = ''.join(takewhile(lambda c: not c.isalpha(), ref))
            ref_out = ''.join(c for c in ref if c.isalnum() or c == '-')
            if not ref_out.isalpha():
                ref_out = ref_out.rstrip(string.ascii_letters)
        return ref_out

r = 'asdfp**10**(?)**'
r = '1*'
print(cleanRef(r))



In [ ]:
ref = ('Book', 'chapter', 'subchapter', 'section', 'line', 'text')
print(ref[:-1])